In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib

database_name = "TSX_Quality.sqlite"

### Utility function to get prices from our local database  

In [2]:
def get_prices_for(symbol):
    conn = sqlite3.connect(database_name)
    
    # Make sur all dates are in ascending order
    sql = f"SELECT * FROM Prices_Daily WHERE Ticker = '{symbol}' ORDER BY UPPER(Ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="Date")
    prices.index = pd.to_datetime(prices.index)
    
    # Clean up CSV data to make sure we have only floats and no "-" values
    prices.replace("-", np.NaN, inplace=True)
    #prices["Volume"].replace(0, np.NaN, inplace=True)
    prices["Open"] = prices["Open"].astype(float)
    prices["High"] = prices["High"].astype(float)
    prices["Low"]  = prices["Low"].astype(float)
    prices["Close"]= prices["Close"].astype(float)
    
    # Required for finTA
    prices.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    prices.index.rename("date", inplace=True) 
    
    return prices

### Indicator calculations added to the prices dataframe

In [6]:
def EMA(prices, days=20, smoothing=2):
    ema = [sum(prices[:days]["close"]) / days]
    for close_price in prices[days:]["close"]:
        new_value = close_price * (smoothing / (1 + days) )
        ema_prev_day = ema[-1]
    return ema

In [7]:
symbol = "SHOP"
prices_df = get_prices_for(symbol)
prices_df

EMA(prices_df, days=20)

[0.04000000000000001]